In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [24]:
class Net(nn.Module): #다른 클래스에서 상속받겠다.
  def __init__(self):
    super().__init__() #인스턴스화 된 net에서 nn.Module을 반복적으로 상속 호출 하지 않기 위한 것
    #입력 이미지 채널 1개 , 출력 채널 6개 3x3 정사각 컨벌루젼 행렬
    #컨벌루젼 커널 정의
    self.conv1 = nn.Conv2d(1, 6, 3)
    self.conv2 = nn.Conv2d(6, 16, 3)
    #어파인 연산 y = Wx + b
    self.fc1 = nn.Linear(16 * 6 * 6, 120)
    self.fc2 = nn.Linear(120, 64)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    # ( 2, 2)크기 윈도우에 대해 맥스 풀링
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    #크기가 제곱수라면 하나의 숫자만을 특정
    x = F.max_pool2d(F.relu(self.conv2d(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

  def num_flat_features(self, x):
    size = x.size()[1:] #배치 차원을 제외한 모든 차원
    num_feature = 1
    for s in size:
      num_feature *= s
    return num_feature


In [25]:
net = Net()
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [26]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

batch_size = 4
epochs = 2

In [27]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle=True, num_workers=2)
#트레인 셋의 순서라던가 몇개 있다던가 그런것을 정의

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
classes = ('plane' ,'car','bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship','truck')

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)

In [30]:
for epoch in range(epochs):
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # input labels의 목록인 data로부터 입력을 받은 후
    inputs, labels = data[0].to(device), data[1].to(device) #데이터들을 gpu로 올리는 코드, device는 단지 gpu로 연결하는 애일 뿐이다.

    #변화도(gradient) 매개변수를 0으로 만들고
    optimizer.zero_grad()

    outputs = net(inputs) #forward 발동
    loss = criterion(outputs, labels) #정답과 테스트를 비교해서 에러율을 찾는다.
    loss.backward() #그 에러를 오차역전파 한다 
    optimizer.step()

    #통계를 출력
    running_loss += loss.item() #item 은 뭐야?
    if i % 2000 == 1999: #print every 2000 mini-batches
      print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 2000))
      running_loss = 0.0
  
  print('finished training')

RuntimeError: ignored

In [ ]:
%